In [ ]:
import os
import torch
print(torch.cuda.is_available())

from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
from dotenv import load_dotenv

load_dotenv("./keys.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")

In [3]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=OPENAI_API_KEY)

In [ ]:
embed_model_name = "intfloat/multilingual-e5-large-instruct"
index_path = "./indexes/2025-04-10-01-02-18"


model_kwargs = {
    "device": "cuda:1",
    "trust_remote_code": True
}
encode_kwargs = {
    "normalize_embeddings": True,
    "batch_size": 128,
    "multi_process": True,
    "show_progress": True
}

embedding_model = HuggingFaceEmbeddings(
    model_name=embed_model_name, 
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vector_store = FAISS.load_local(index_path, embeddings=embedding_model, allow_dangerous_deserialization=True)

In [ ]:
"""
reference : https://wikidocs.net/234016 

vectore_store에 대한 검색기(retriever)를 생성.
 - search_type : 검색 유형. similarity, mmr(Maximal Marginal Relevance), similarity_score_threshold 중 1개
 - search_kwargs : 추가 검색 옵션
    - k : 반환할 문서 수(기본=4)
    - score_threshold : 유사도 임계치(기본=0.0)
    - fetch_k : MMR 알고리즘에 전달할 문서 수(기본=20)
    - lambda_mult : MMR 결과의 다양성을 조절.(0~1사이 값으로 기본값=0.5)
    - filter : 문서 메타데이터 기반 필터링 --> 메타데이터 필터링으로 특정 조건의 문서만 검색 가능.
"""

retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 10, "score_threshold": 0.3}
)

ret_results = retriever.invoke("코리아교육그룹의 비전, 미래, 목표")

for ret in ret_results:
    print(ret.metadata['company_name'], ret.metadata['url'], ret.metadata['original_doc_id'])
    print("="*100)
    print(ret.page_content)

In [7]:
from langgraph.graph import StateGraph, MessagesState

graph_builder = StateGraph(MessagesState)

In [8]:
from langchain_core.tools import tool

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [9]:
def query_or_respond(state:MessagesState):
    """검색 툴을 호출하거나 직접 응답을 생성"""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])

    return {"messages": [response]}

In [10]:
from langgraph.prebuilt import ToolNode

tools = ToolNode([retrieve])

In [11]:
from langchain_core.messages import SystemMessage

def generate(state:MessagesState):
    """응답을 생성하는 노드"""
    print("\n===== state['messages'] 내용 출력 =====")
    for i, msg in enumerate(state["messages"]):
        print(f"[{i}] type: {msg.type}, content: {msg.content}")
    print("====================================\n")

    recent_tool_messages = [] ## 생성된 ToolMessage를 저장할 리스트
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1] ## 역순으로 정렬
    docs_content = "\n\n".join(doc.content for doc in tool_messages) ## 검색된 문서들을 프롬프트에 반영할 수 있도록 문자열로 변환

    ## 시스템 프롬프트
    system_message_content = (
        "당신은 주어진 질문에 대해 정확한 답변을 제공하는 전문가입니다."
        "다음은 주어진 질문과 검색된 컨텍스트입니다."
        "이 컨텍스트를 사용하여 질문에 대한 정확한 답변을 제공하세요."
        "만약 답변을 알 수 없다면, '알 수 없습니다'라고 답변하세요."
        "\n\n"
        f"{docs_content}"
    )

    ## 대화 메시지 추출
    conversation_messages = []
    for message in state["messages"]:
        ## 사용자나 시스템 메세지 포함
        if message.type in ("human", "system"):
            conversation_messages.append(message)
        
        ## AI 메세지이고 툴 호출이 없는 경우 포함
        elif message.type == "ai" and not message.tool_calls:
            conversation_messages.append(message)
    
    ## 시스템 메시지와 대화 메시지 결합
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [12]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges("query_or_respond", tools_condition, {END:END, "tools":"tools"})
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
input_message = "안녕하세요"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "코리아교육그룹의 비전, 목표, 미래에 대한 내용들을 추출하고 정리해서 문장형태로 서술해주세요."

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [21]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
config = {"configurable" : {"thread_id" : "abc123"}}

In [ ]:
input_message = "코리아교육그룹의 비전, 목표, 미래에 대한 내용들을 추출하고 정리해서 문장형태로 서술해주세요."

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()


In [ ]:
input_message = "코리아교육그룹의 기업문화, 업무스타일에 대해 알려줘"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()


In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
config = {"configurable" : {"thread_id" : "abc123"}}

agent_executor = create_react_agent(llm, [retrieve], checkpointer=memory)

display(Image(agent_executor.get_graph().draw_mermaid_png()))

In [ ]:
input_message = (
    "코리아교육그룹의 비전, 목표, 미래에 대한 내용들을 추출하고 정리해서 문장형태로 서술해주세요.\n\n"
    "서술된 내용을 바탕으로 코리아교육그룹이 비전을 이루기 위해 어떤 노력을 하는지, 어떤 문화를 만들었는지 설명해줘."
)

for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "주어진 기업이 설정한 비전, 목표, 미래에 대한 내용들을 추출하고 정리해서 문장형태로 서술해주세요."
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{input_documents}")
])

str_ret_results = [ret.page_content for ret in ret_results]
prompt = prompt_template.invoke({"input_documents": str_ret_results})
print(prompt)

In [ ]:
from langchain_anthropic import ChatAnthropic

chat_model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0, api_key=ANTHROPIC_API_KEY)
response = chat_model.invoke(prompt)
print(response.content)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnableLambda

company_name = "LG CNS"

# 1. 시스템 프롬프트 템플릿 정의
system_template = "주어진 기업이 설정한 비전, 목표, 미래에 대한 내용들을 추출하고 정리해서 문장형태로 서술해주세요."
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{input_documents}")
])

# 2. 벡터 저장소 → retriever (Runnable)
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 10, "score_threshold": 0.3}
)

# 3. 검색 결과에서 page_content만 추출하는 함수 (Runnable로 래핑)
extract_content = RunnableLambda(lambda docs: {"input_documents": [doc.page_content for doc in docs]})

# 4. LLM 모델
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0, api_key=ANTHROPIC_API_KEY)

# 5. 체인 구성: retriever → page_content 추출 → prompt → llm
chain = retriever | extract_content | prompt_template | llm

# 6. 실행
response = chain.invoke(f"{company_name}의 비전, 미래, 목표")
print(f"{company_name}에 대한 응답:\n{response.content}")
